I. Importing Python libraries that are applicable in the following essay. 

In this essay, a number of libraries are used besides ModSim, including control, numpy, scipy and matplotlib.pyplot. The existing functions in these imported libraries make the code clearer and more succint. 

In [1]:
# Configure Jupyter so figures appear in the notebook
%matplotlib inline

# Configure Jupyter to display the assigned value after an assignment
%config InteractiveShell.ast_node_interactivity='last_expr_or_assign'

# import functions from the modsim.py module
from modsim import *

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

import control as cntrl
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'control'

II. Defining system

In this section, the system (the value of the inductor, capacitor and resistor) are being defined. By running make_system, a set of system value is updated. 

In [5]:
def make_system(L, C, R): 
    
#L = 4.7e-5
#C = 4.7e-5
#R = 1e3


    sim_timestep = 0.01
    return System(L = L, C = C, R = R, sim_timestep = sim_timestep)
system = make_system(4.7e-5, 4.7e-5,1e3)

,values
L,0.000047
C,0.000047
R,1000.000000
sim_timestep,0.010000


In [6]:
top = np.asarray([1])
bottom = np.asarray([system.L*system.C, system.R*system.C, 1])

array([2.209e-09, 4.700e-02, 1.000e+00])

In [ ]:
sys_tf = cntrl.tf(top, bottom)
sys_ss = cntrl.tf2ss(sys_tf)
sys_ss_d = cntrl.c2d(sys_ss, sim_timestep)
np.zeros((sys_ss_d.B.shape[0], 1))

In [ ]:
def step(sys, mag=1.0, time=5):
    state = np.zeros((sys.A.shape[0], 1))
    dt = sys.dt
    t = []
    y = []
    for i in range(int(time/dt)):
        state = np.matmul(sys.A, state) + sys.B*mag
        output = np.matmul(sys.C, state)
        t.append(i*dt)
        y.append(float(output))
    return t,y

In [ ]:
t, y = step(sys_ss_d, mag=1.0, time=1)

In [ ]:
plt.plot(t, y)
plt.legend("V")
plt.xlabel("Time (s)")
plt.ylabel("Voltage (v)")
plt.title("RLC filter step response")
plt.show()

In [ ]:
def lsim(sys, vec):
    state = np.zeros((sys.A.shape[0], 1))
    dt = sys.dt
    t = []
    y = []
    for i in range(len(vec)):
        state = np.matmul(sys.A, state) + sys.B*vec[i]
        output = np.matmul(sys.C, state)
        t.append(i*dt)
        y.append(float(output))
    return t,y

In [ ]:
sin_x = np.linspace(0, 8*np.pi, 100)
sin_y = np.sin(sin_x)
t, y = lsim(sys_ss_d, sin_y)
sin_x /= 8*np.pi

In [ ]:
plt.plot(t, y, sin_x, sin_y)
plt.xlabel("Time (s)")
plt.ylabel("Voltage (v)")
plt.title("RLC filter sine response")
plt.show()